LangChain includes functionality to integrate directly with Neo4j, including allowing you to run Cypher statements, query vector indexes and use Neo4j as a conversation memory store.

LangChain, Cypher ifadelerini çalıştırmanıza, vektör dizinlerini sorgulamanıza ve Neo4j'i bir konuşma belleği deposu olarak kullanmanıza olanak tanıyan Neo4j ile doğrudan entegre olma işlevselliğine sahiptir.

### Connecting to a Neo4j instance


In [ ]:
#Aşağıdaki kod bir NEO4J veritabanına bağlanır ve basit bir sorgu çalıştırır.

import os
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD")
)

result = graph.query("""
MATCH (m:Movie{title: 'Toy Story'}) 
RETURN m.title, m.plot, m.poster
""")

print(result)

Schema
NEO4J veritabanına bağlandığınızda, nesne veritabanı şemasını belleğe yükler - bu, Langchain'in veritabanını sorgulamak zorunda kalmadan şema bilgilerine erişmesini sağlar.
Şema bilgilerine şema özelliğini kullanarak erişebilirsiniz.

print(graph.schema)

Refreshing the schema:
You can refresh the schema by calling the graph.refresh_schema() method.



### Conversation History
Konuşma geçmişi
In the previous lesson, you created a program that used the ChatMessageHistory component to store conversation history in memory.
Önceki derste, konuşma geçmişini bellekte saklamak için chatMessageHistory bileşenini kullanan bir program oluşturdunuz.

You will now update this program to store the conversation history in your Neo4j sandbox using the Neo4jChatMessageHistory component.
Şimdi Neo4jchatMessageHistory bileşenini kullanarak NEO4J sanal alanınızda konuşma geçmişini saklamak için bu programı güncelleyeceksiniz.


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        ("system", "{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

memory = ChatMessageHistory()

def get_memory(session_id):
    return memory

chat_chain = prompt | chat_llm | StrOutputParser()

chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_memory,
    input_messages_key="question",
    history_messages_key="chat_history",
)


current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

while (question := input("> ")) != "exit":

    response = chat_with_message_history.invoke(
        {
            "context": current_weather,
            "question": question,
            
        }, 
        config={
            "configurable": {"session_id": "none"}
        }
    )
    
    print(response)

### Session ID - Oturum Kimliği
You must create and assign a session ID to each conversation to identify them.
Bunları tanımlamak için her konuşmaya bir oturum kimliği oluşturmalı ve atamalısınız.

The session ID can be any unique value, such as a Universally Unique Identifier (UUID).
Oturum kimliği, evrensel olarak benzersiz bir tanımlayıcı (UUID) gibi benzersiz bir değer olabilir.

You can generate a random UUID using the Python uuid.uuid4 function.
Python UUID.UUID4 işlevini kullanarak rastgele bir UUID oluşturabilirsiniz.

Create a new SESSION_ID constant in your chat model program.
Sohbet modeli programınızda yeni bir oturum_id sabiti oluşturun.

In [ ]:
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")
# Bu oturum kimliği, Neo4J'deki konuşmayı tanımlamak için kullanılacaktır.

### Neo4j Chat Message History
Neo4j sohbet mesaj geçmişi
Create a Neo4jGraph object to connect to your Neo4j sandbox.
Neo4j kum havuzunuza bağlanmak için bir neo4jgraph nesnesi oluşturun.

In [ ]:
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD")
)

Previously, the get_memory function returned an instance of ChatMessageHistory.
Daha önce, get_memory işlevi bir chatMessageHistory örneği döndürmüştü.

The get_memory function should now return an instance of Neo4jChatMessageHistory. You should pass the session_id and the graph connection you created as parameters.
Get_Memory işlevi artık Neo4jchatMessageHistory örneğini döndürmelidir. Session_id'i ve parametre olarak oluşturduğunuz grafik bağlantısını geçmelisiniz.


In [ ]:
from langchain_neo4j import Neo4jChatMessageHistory

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

Finally, you must add the SESSION_ID to the config when you invoke the chat model.
Son olarak, sohbet modelini çağırdığınızda seans_id'i yapılandırmaya eklemelisiniz.

In [ ]:
    response = chat_with_message_history.invoke(
        {
            "context": current_weather,
            "question": question,
            
        }, 
        config={
            "configurable": {"session_id": SESSION_ID}
        }
    )

compalte code
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_neo4j import Neo4jGraph
from langchain_neo4j import Neo4jChatMessageHistory
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

chat_llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD")
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a surfer dude, having a conversation about the surf conditions on the beach. Respond using surfer slang.",
        ),
        ("system", "{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

chat_chain = prompt | chat_llm | StrOutputParser()

chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_memory,
    input_messages_key="question",
    history_messages_key="chat_history",
)

current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

while (question := input("> ")) != "exit":
    
    response = chat_with_message_history.invoke(
        {
            "context": current_weather,
            "question": question,
            
        }, 
        config={
            "configurable": {"session_id": SESSION_ID}
        }
    )
    
    print(response)

Aşağıdaki Cypher sorgusunu kullanarak konuşma geçmişinin grafiğini döndürebilirsiniz

MATCH (s:Session)-[:LAST_MESSAGE]->(last:Message)<-[:NEXT*]-(msg:Message)
RETURN s, last, msg

Session.id özelliğine filtreleyerek tek bir oturum için konuşma geçmişini döndürebilirsiniz.

MATCH (s:Session)-[:LAST_MESSAGE]->(last:Message)
WHERE s.id = 'your session id'
MATCH p = (last)<-[:NEXT*]-(msg:Message)
UNWIND nodes(p) as msgs
RETURN DISTINCT msgs.type, msgs.content

Doğru olan Langchain Neo4j entegrasyonu hakkında tüm ifadeleri seçin.

 You can connect to a Neo4j databaseNeo4J veritabanına bağlanabilirsiniz
 The database schema is loaded automaticallyVeritabanı şeması otomatik olarak yüklenir
 You can run Cypher queriesCypher sorgularını çalıştırabilirsiniz
 Vectors can be queriedVektörler sorgulanabilir
 You can use Neo4j as a conversation memory storeNeo4j'i konuşma bellek mağazası olarak kullanabilirsiniz